In [1]:
import numpy as np
import pandas as pd
import random
import glob
from sklearn import preprocessing


In [23]:
# Get list of files for the Geo grouped data
data_files_list = glob.glob('/datasets/cms/MUP*.csv')
data_files_list

# For training use 2013 to 2016 to predict 2017
# For testing use 2014 to 2017 to predict 2018
# for validation use 2015 to 2018 to predict 2019

# For presentation use 2016 to 2019 to predict 2020 (and compare against 2019)

['/datasets/cms/MUP_PHY_R19_P04_V10_D13_Geo.csv',
 '/datasets/cms/MUP_PHY_R19_P04_V10_D14_Geo.csv',
 '/datasets/cms/MUP_PHY_R19_P04_V10_D15_Geo.csv',
 '/datasets/cms/MUP_PHY_R19_P04_V10_D16_Geo.csv',
 '/datasets/cms/MUP_PHY_R19_P04_V10_D17_Geo.csv',
 '/datasets/cms/MUP_PHY_R20_P04_V10_D18_Geo.csv',
 '/datasets/cms/MUP_PHY_R21_P04_V10_D19_Geo.csv']

In [35]:
df_geo_full = []
for _file in data_files_list:
    df_geo = pd.read_csv(
         _file,
         header=0, 
         #skiprows=lambda i: i>0 and random.random() > p
    )
    year= _file[-10:-8]
    df_geo['year'] = 2000+int(year)
    if len(df_geo_full)!=0:
        df_geo_full = df_geo_full.append(df_geo, ignore_index=False)
    else:
        df_geo_full = df_geo.copy()

df_geo_full= df_geo_full.loc[df_geo_full['year']!= 0,:]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
# Prep geo_full for train/test/val

df_geo_features = df_geo_full.loc[:,['Rndrng_Prvdr_Geo_Desc', 'HCPCS_Cd', 'Tot_Srvcs','year']].copy()
df_geo_features['idx'] = 

In [51]:
df_geo_features = df_geo_features.groupby(['Rndrng_Prvdr_Geo_Desc', 'HCPCS_Cd','year'])[ 'Tot_Srvcs'].sum().reset_index()

df_geo_features.loc[:,'idx'] = df_geo_features.apply(lambda x:  "_".join([str(x['Rndrng_Prvdr_Geo_Desc']), str(x['HCPCS_Cd'])]), axis=1)
df_geo_features.pivot(index=['idx'], columns=['year'], values='Tot_Srvcs')

In [52]:
df_geo_features.sample(5)

,Rndrng_Prvdr_Geo_Desc,HCPCS_Cd,year,Tot_Srvcs
698041,Mississippi,J2250,2015,676.0
120424,California,45379,2016,119.0
388079,Illinois,J3475,2016,39846.0
40305,Alaska,96127,2019,90.0
345784,Idaho,77059,2016,173.0


year,2013,2014,2015,2016,2017,2018,2019
count,2.072850e+05,2.077600e+05,2.086830e+05,2.087350e+05,2.089420e+05,2.106080e+05,2.114710e+05
mean,2.436985e+04,2.555518e+04,2.706354e+04,2.751802e+04,2.835129e+04,2.943969e+04,3.065268e+04
std,6.547964e+05,6.860724e+05,7.197823e+05,7.212535e+05,7.314109e+05,7.489473e+05,7.626620e+05
min,9.500000e+00,1.100000e+01,1.100000e+01,1.100000e+01,1.100000e+01,1.100000e+01,1.100000e+01
25%,4.400000e+01,4.400000e+01,4.400000e+01,4.400000e+01,4.412500e+01,4.500000e+01,4.600000e+01
50%,1.880000e+02,1.860000e+02,1.880000e+02,1.870000e+02,1.890000e+02,1.910000e+02,2.000000e+02
75%,1.302000e+03,1.309250e+03,1.332000e+03,1.335000e+03,1.357000e+03,1.392000e+03,1.474000e+03
max,1.407623e+08,1.433992e+08,1.416137e+08,1.406367e+08,1.411027e+08,1.380397e+08,1.345132e+08


In [57]:
df_geo_features.pivot(index=['idx'], columns=['year'], values='Tot_Srvcs').sample(5)

year,2013,2014,2015,2016,2017,2018,2019
idx,,,,,,,
New York_Q4101,7615.0,4415.0,8212.0,7019.0,5571.0,2995.0,5909.0
Connecticut_86078,170.0,135.0,151.0,123.0,112.0,111.0,102.0
Utah_85555,20.0,NaN,NaN,NaN,NaN,NaN,NaN
Wisconsin_70549,2098.0,1960.0,2021.0,2050.0,1992.0,1623.0,1482.0
Tennessee_57100,137.0,160.0,114.0,139.0,144.0,137.0,133.0


In [49]:
df_geo_features.sample(5)

Tot_Srvcs                                  \
year                                2013    2014    2015    2016    2017   
Rndrng_Prvdr_Geo_Desc HCPCS_Cd                                             
Missouri              76497          NaN     NaN     NaN     NaN     NaN   
Nebraska              57282        157.0   116.0   139.0   106.0    79.0   
North Carolina        95982         18.0     NaN     NaN     NaN     NaN   
New York              50949         15.0     NaN    23.0    32.0    38.0   
Virginia              88334        832.0  1039.0  1047.0  1122.0  1193.0   

                                               
year                              2018   2019  
Rndrng_Prvdr_Geo_Desc HCPCS_Cd                 
Missouri              76497       12.0   13.0  
Nebraska              57282       95.0   77.0  
North Carolina        95982        NaN    NaN  
New York              50949       58.0   44.0  
Virginia              88334     1070.0  972.0

In [6]:
# Import population estimates data
pop_est = pd.read_csv("/datasets/cms/SC-EST2020-ALLDATA5.csv")

# Drop columns that is not needed for "one row" melting
pop_est.drop(['SUMLEV', 'REGION', 'DIVISION', 'STATE'], axis='columns', inplace=True)

In [7]:
pop_est.sample(3)

,NAME,SEX,ORIGIN,RACE,AGE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020
189442,West Virginia,2,2,3,70,1,1,1,2,0,0,2,1,2,1,2,2,4
117978,New Jersey,1,1,2,72,2353,2353,2339,2325,2494,2487,2571,2808,2768,2771,2696,3308,3203
22760,Colorado,2,1,5,56,43,43,49,60,53,63,67,75,88,90,75,90,97


In [8]:
pop_est['age_category'] =  pop_est['AGE'].apply(lambda x: 1 if x >40  else 0) 
#pop_est['idx'] = pop_est.apply(lambda x:  "_".join([str(x['SEX']), str(x['ORIGIN']), str(x['RACE']), str(x['AGE'])]), axis=1)
pop_est.drop(['SEX', 'ORIGIN', 'RACE', 'AGE'], axis='columns', inplace=True)

In [18]:
pop_agg_df = pop_est.copy()
pop_agg_df.drop(['CENSUS2010POP','ESTIMATESBASE2010'], axis = 'columns', inplace=True)

pop_agg_df = pd.melt(pop_agg_df,id_vars=['NAME','age_category'])

pop_agg_df.loc[:,'year'] = pop_agg_df['variable'].apply(lambda x: int(x[-4:]))
#popestimate df
pop_agg_df.drop(['variable'], axis='columns', inplace=True)

pop_agg_df = pop_agg_df.groupby(['NAME','age_category','year']).sum().reset_index()
# set t=0 for base year

# set year count to integers from 0 to x for train and test split
max_year = 2017
pop_agg_df.loc[:,'year'] = max_year - pop_agg_df['year'] 
pop_train_df = pop_agg_df.loc[pop_agg_df['year']>0,:].copy()
max_train_year = max(pop_train_df['year'])
pop_test_df = pop_agg_df.loc[pop_agg_df['year']>-1,:].copy()
pop_test_df.loc[:,'year'] = pop_test_df['year'] + 1
pop_test_df = pop_test_df.loc[pop_test_df['year'] <= max_train_year,:]

pop_train_df.loc[:,'idx'] = pop_train_df.apply(lambda x:  "_".join([str(x['age_category']), str(x['year'])]), axis=1)
pop_train_df = pop_train_df.pivot(index='NAME', columns=['idx'], values='value')

pop_test_df.loc[:,'idx'] = pop_test_df.apply(lambda x:  "_".join([str(x['age_category']), str(x['year'])]), axis=1)
pop_test_df = pop_test_df.pivot(index='NAME', columns=['idx'], values='value')

# normalise values based on train data only

_mean = pop_train_df.stack().mean()
_std = pop_train_df.stack().std()

pop_train_df = (pop_train_df-_mean)/_std
pop_test_df = (pop_test_df-_mean)/_std

In [19]:
pop_train_df.sample(3)

idx,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7
NAME,,,,,,,,,,,,,,
Montana,-0.719805,-0.721615,-0.723283,-0.724850,-0.726739,-0.727490,-0.728352,-0.732411,-0.733775,-0.734741,-0.735731,-0.736841,-0.738089,-0.739268
North Carolina,0.658788,0.643588,0.633484,0.627539,0.622131,0.620438,0.619908,0.448538,0.426964,0.407335,0.386330,0.363548,0.337631,0.312946
Utah,-0.311470,-0.321015,-0.327587,-0.332821,-0.339469,-0.344480,-0.349513,-0.577485,-0.585620,-0.592283,-0.598628,-0.604989,-0.611519,-0.617953


In [21]:
pop_test_df.describe()

idx,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,0.106973,0.101994,0.096615,0.090393,0.085140,0.081423,0.079016,-0.052798,-0.060326,-0.069010,-0.076945,-0.085678,-0.095134,-0.106482
std,1.156042,1.150450,1.143471,1.133757,1.125057,1.119097,1.114259,0.913021,0.902694,0.890544,0.878645,0.866043,0.853253,0.838481
min,-0.788075,-0.788208,-0.787871,-0.787886,-0.787597,-0.787414,-0.786682,-0.799485,-0.799653,-0.800396,-0.801277,-0.802269,-0.803536,-0.804873
25%,-0.585639,-0.587698,-0.591267,-0.593764,-0.596479,-0.598230,-0.599249,-0.644942,-0.646355,-0.648357,-0.649853,-0.651699,-0.653852,-0.656490
50%,-0.208054,-0.210387,-0.211169,-0.212161,-0.211738,-0.212692,-0.211955,-0.293755,-0.296105,-0.300066,-0.303727,-0.307325,-0.312142,-0.318035
75%,0.273359,0.255898,0.237569,0.222288,0.208574,0.200355,0.193412,0.057945,0.048063,0.037267,0.027393,0.017033,0.006083,-0.007232
max,5.531711,5.517796,5.498762,5.461994,5.426293,5.400731,5.385038,3.980014,3.931768,3.872971,3.810965,3.743690,3.672994,3.592945


In [ ]:
pop_agg_df['value'][1:].astype(float).reset_index().divide(pop_agg_df['value'][0:-1].astype(float))

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
pop_agg_df['value'][1:-1].astype(float)

1          40882.0
2          39995.0
3          38930.0
4          39839.0
5          40054.0
            ...   
2171064        3.0
2171065        4.0
2171066        3.0
2171067        5.0
2171068        3.0
Name: value, Length: 2171068, dtype: float64

In [ ]:
pop_agg_df.head(10)

,NAME,idx,value,year
0,Alabama,0_0_1_0,39928,2010
1,Alabama,0_0_1_0,40882,2011
2,Alabama,0_0_1_0,39995,2012
3,Alabama,0_0_1_0,38930,2013
4,Alabama,0_0_1_0,39839,2014
5,Alabama,0_0_1_0,40054,2015
6,Alabama,0_0_1_0,40221,2016
7,Alabama,0_0_1_0,39345,2017
8,Alabama,0_0_1_0,39036,2018
9,Alabama,0_0_1_0,37587,2019


In [ ]:
val_columns = [ x for x in pop_est.columns if x not in ['idx','NAME']] 
pop_est_pivot = pop_est.pivot_table(index=['NAME'], columns='idx', 
                    values=val_columns, 
                    aggfunc='first')

In [ ]:
pop_est.sample(5)

CENSUS2010POP                                              \
idx                 0_0_1_0 0_0_1_1 0_0_1_10 0_0_1_11 0_0_1_12 0_0_1_13   
NAME                                                                      
New Hampshire         12220   12762    15751    15727    16194    16336   
Maryland              42563   42664    45042    45052    44536    45508   
Missouri              63552   63753    67011    66154    66169    65576   
Oklahoma              40014   40422    39598    39075    38659    37744   
Colorado              59926   60766    61670    59936    58921    58959   

                                                   ... POPESTIMATE2020  \
idx           0_0_1_14 0_0_1_15 0_0_1_16 0_0_1_17  ...        2_2_5_78   
NAME                                               ...                   
New Hampshire    16905    17298    17577    18101  ...               0   
Maryland         46525    47137    47107    48867  ...               3   
Missouri         66387    66842    67640    68705  ...               7   
Oklahoma         37680    37861    38508    39301  ...               2   
Colorado         58657    59199    59603    60378  ...               4   

                                                                             \
idx           2_2_5_79 2_2_5_8 2_2_5_80 2_2_5_81 2_2_5_82 2_2_5_83 2_2_5_84   
NAME                                                                          
New Hampshire        0       7        0        2        0        0        0   
Maryland             5     113        2        2        2        4        2   
Missouri             0      14        0        1        1        0        0   
Oklahoma             3      21        0        1        1        0        0   
Colorado             2      57        3        1       12        3        0   

                                
idx           2_2_5_85 2_2_5_9  
NAME                            
New Hampshire        0       4  
Maryland             8      83  
Missouri             2      21  
Oklahoma             4       7  
Colorado            15      58  

[5 rows x 50310 columns]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f3594ae2-e1b0-4476-8420-ffda39dfce6e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>